<a href="https://colab.research.google.com/github/Haebuk/kuggle/blob/main/gamestop_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, InputLayer
from sklearn.model_selection import train_test_split

PATH = '/content/drive/MyDrive/input/'
df = pd.read_csv(PATH + 'GME_scaled.csv')

hpo_configs = {
    'learning_rate': 0.006008989711552188,
    'dropout_rate1': 0.39661787877288335,
    'hidden1': 15,
    'time_step': 4,
}

X = df.drop('date', axis=1).values
y = df['open_price'].values
X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.1, shuffle=False)

time_step = hpo_configs['time_step']
X_train, y_train, X_test, y_test = [], [], [], []
for i in range(time_step,len(X_tr)): # train
    X_train.append(X_tr[i-time_step:i,:])
    y_train.append(y_tr[i])
for i in range(time_step, len(X_te)): # test
    X_test.append(X_te[i-time_step:i,:])
    y_test.append(y_te[i])
X_train, y_train = np.array(X_train), np.array(y_train)
X_test, y_test = np.array(X_test), np.array(y_test)
# LSTM input shape 조건(batch_size, timestep, feature_num)에 맞게 reshape
X_train = np.reshape(X_train, (-1,X_train.shape[1],2))
X_test = np.reshape(X_test, (-1,X_test.shape[1],2))

model = tf.keras.Sequential([
InputLayer(input_shape=(hpo_configs['time_step'], 2)),
LSTM(hpo_configs['hidden1'], return_sequences=True),
Dropout(hpo_configs['dropout_rate1']),
Dense(1),
])

opt = tf.keras.optimizers.Adam(learning_rate=hpo_configs['learning_rate'])

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath = 'best-model-weights.h5',
    save_weights_only=True,
    save_best_only=True,
    monitor='val_loss',
    verbose=1
)

model.compile(optimizer=opt,
            loss='mse')

model.fit(X_train, y_train,
        validation_data=(X_test, y_test),
        epochs=200, callbacks=[early_stopping, checkpoint])

Epoch 1/200
82/82 [==============================] - 5s 8ms/step - loss: 1.5772 - val_loss: 1.1386

Epoch 00001: val_loss improved from inf to 1.13857, saving model to best-model-weights.h5
Epoch 2/200
82/82 [==============================] - 0s 4ms/step - loss: 0.4447 - val_loss: 0.5248

Epoch 00002: val_loss improved from 1.13857 to 0.52477, saving model to best-model-weights.h5
Epoch 3/200
82/82 [==============================] - 0s 3ms/step - loss: 0.3526 - val_loss: 0.4144

Epoch 00003: val_loss improved from 0.52477 to 0.41439, saving model to best-model-weights.h5
Epoch 4/200
82/82 [==============================] - 0s 3ms/step - loss: 0.3255 - val_loss: 0.4459

Epoch 00004: val_loss did not improve from 0.41439
Epoch 5/200
82/82 [==============================] - 0s 4ms/step - loss: 0.2939 - val_loss: 0.3229

Epoch 00005: val_loss improved from 0.41439 to 0.32291, saving model to best-model-weights.h5
Epoch 6/200
82/82 [==============================] - 0s 4ms/step - loss: 0.25

In [3]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 4, 15)             1080      
_________________________________________________________________
dropout (Dropout)            (None, 4, 15)             0         
_________________________________________________________________
dense (Dense)                (None, 4, 1)              16        
Total params: 1,096
Trainable params: 1,096
Non-trainable params: 0
_________________________________________________________________


In [7]:
model.load_weights('best-model-weights.h5')

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 4, 15)             1080      
_________________________________________________________________
dropout (Dropout)            (None, 4, 15)             0         
_________________________________________________________________
dense (Dense)                (None, 4, 1)              16        
Total params: 1,096
Trainable params: 1,096
Non-trainable params: 0
_________________________________________________________________


# Inference

In [ ]:
# evaluate, predict, graph